In [26]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI(
    base_url=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY"),
)

response = client.chat.completions.create(
    model="openchat/openchat-7b",
    messages=[
        {"role": "system", "content": "You are a helpful AI fitness assistant."},
        {"role": "user", "content": "Give me 3 good chest exercises and how many calories I’ll burn doing them for 30 minutes. I weigh 75kg. Please be concise only give 1 sentence answer per exercise"}
    ]
)

print(response.choices[0].message.content)


Bench Press: Aids in strength & muscle hypertrophy
15 Calories Burned per min = 120min calories burned total; Pushups: 0 Cal per minute; Cardio: Cyclic movement engaging cardiovascularly
82 Cal per minute = 783 calorie burnt total ;Plank : .09 per minute burn of calories or burning 3.6 for half an hour while maintaining


In [27]:
from langchain.tools import tool

@tool
def estimate_calories(exercise: str, duration_min: int, weight_kg: float) -> str:
    """This function is used to calculate calories for an exercise. Input the Exercise as string and the duration and the person weight to calculate."""
    met_values = {
        "push ups": 3.8,
        "bench press": 6.0,
        "dumbbell flyes": 4.0,
    }
    met = met_values.get(exercise.lower(), 4.0)
    calories = (met * 3.5 * weight_kg / 200) * duration_min
    return f"Estimated {calories:.1f} calories burned doing {exercise} for {duration_min} mins at {weight_kg}kg."


In [76]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model="google/gemini-2.0-flash-001",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE"),
    temperature=0,
)

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
openrouter_chain = prompt | llm
print(openrouter_chain.invoke({"topic": "banana"}))


content="Why did the banana go to the doctor?\n\nBecause it wasn't peeling well!\n" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 7, 'total_tokens': 26, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'google/gemini-2.0-flash-001', 'system_fingerprint': None, 'id': 'gen-1744094250-bj2nmf06JYUjCMP1ukvV', 'finish_reason': 'stop', 'logprobs': None} id='run-555d2732-ff0e-4a10-b895-3d2f9748076c-0' usage_metadata={'input_tokens': 7, 'output_tokens': 19, 'total_tokens': 26, 'input_token_details': {}, 'output_token_details': {}}


In [77]:
from langchain.agents import initialize_agent, AgentType
from langchain import hub

tools = [estimate_calories]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [78]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm,tools,prompt)

In [79]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [80]:
agent_executor.invoke({"input": "hi there!"})



> Entering new AgentExecutor chain...
Hi there! How can I help you?


> Finished chain.


{'input': 'hi there!', 'output': 'Hi there! How can I help you?\n'}